In [1]:
import math
from math import radians, cos, sin, asin, acos, sqrt, pi
import pandas as pd
import numpy as np
from datetime import time
import datetime as dt
from datetime import datetime as datetime
from datetime import datetime as datetime
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.interpolate import interp1d
import seaborn as sns

In [35]:
# Loading Cleaned data
data_info_clean = pd.read_excel(r"C:\Users\deepak\Desktop\Datathon\InfoVisionaries_Cadiorespiratory_CleaningReport.xlsx",sheet_name='data_info_clean')
data_info_clean
data_test = pd.read_excel(r"C:\Users\deepak\Desktop\Datathon\InfoVisionaries_Cadiorespiratory_CleaningReport.xlsx",sheet_name='data_test_clean')
data_test

,ID,time,RR,VO2,power,HR
0,1,-130.840,672.0,0.302008,0,89.285714
1,1,-130.168,672.0,0.302008,0,89.285714
2,1,-129.536,632.0,0.301336,0,94.936709
3,1,-109.280,672.0,0.310178,0,89.285714
4,1,-104.280,660.0,0.329340,0,90.909091
...,...,...,...,...,...,...
47896,18,947.288,536.0,0.507071,0,111.940299
47897,18,947.816,528.0,0.507071,0,113.636364
47898,18,948.344,528.0,0.507071,0,113.636364
47899,18,948.884,540.0,0.507071,0,111.111111


In [37]:
data_info_clean

,ID,age,Weight,Height,P_vt1,P_vt2,sport,HRmax
0,1,15,53.7,161,50,140,fencing,192.307692
1,2,15,56.2,162,80,127,fencing,192.307692
2,3,16,89.9,191,155,300,fencing,182.926829
3,4,16,76.5,180,215,275,fencing,192.307692
4,5,17,76.4,186,110,172,fencing,189.873418
5,6,17,65.6,169,131,186,fencing,192.307692
6,7,15,49.8,165,65,108,fencing,189.873418
7,8,15,51.2,165,80,133,fencing,192.307692
8,9,14,57.1,176,50,125,fencing,189.873418
9,10,14,58.8,177,101,162,fencing,192.307692


In [48]:
# Dropping HR Max column
data_info_clean = data_info_clean.drop('HRmax',axis=1)
data_info_clean

,ID,age,Weight,Height,P_vt1,P_vt2,sport
0,1,15,53.7,161,50,140,fencing
1,2,15,56.2,162,80,127,fencing
2,3,16,89.9,191,155,300,fencing
3,4,16,76.5,180,215,275,fencing
4,5,17,76.4,186,110,172,fencing
5,6,17,65.6,169,131,186,fencing
6,7,15,49.8,165,65,108,fencing
7,8,15,51.2,165,80,133,fencing
8,9,14,57.1,176,50,125,fencing
9,10,14,58.8,177,101,162,fencing


In [57]:

# Calculate rolling mean for 'HR' column grouped by 'ID'. 
# Window size mentioed as 5 because we are calculating mean value of 5 consecutive HR/VO2 values
window_size = 5

data_test['HR_mean'] = data_test.groupby('ID')['HR'].rolling(window=window_size, min_periods=1).mean().reset_index(level=0, drop=True)

# Calculate rolling mean for 'VO2' column grouped by 'ID'
data_test['VO2_mean'] = data_test.groupby('ID')['VO2'].rolling(window=window_size, min_periods=1).mean().reset_index(level=0, drop=True)

data_test

,ID,time,RR,VO2,power,HR,HR_mean,VO2_mean
0,1,-130.840,672.0,0.302008,0,89.285714,89.285714,0.302008
1,1,-130.168,672.0,0.302008,0,89.285714,89.285714,0.302008
2,1,-129.536,632.0,0.301336,0,94.936709,91.169379,0.301784
3,1,-109.280,672.0,0.310178,0,89.285714,90.698463,0.303882
4,1,-104.280,660.0,0.329340,0,90.909091,90.740589,0.308974
...,...,...,...,...,...,...,...,...
47896,18,947.288,536.0,0.507071,0,111.940299,113.483928,0.481351
47897,18,947.816,528.0,0.507071,0,113.636364,112.955387,0.487781
47898,18,948.344,528.0,0.507071,0,113.636364,113.126269,0.494211
47899,18,948.884,540.0,0.507071,0,111.111111,112.792100,0.500641


In [58]:
# calculating max of HR and VO2 from the Mean HR/VO2 values by ID

def max_na_rm(series):
    return series.dropna().max()

# Group by 'ID' and calculate maximum values
result = data_test.groupby('ID').agg(
    HRmax=('HR_mean', max_na_rm),
    VO2max_abs=('VO2_mean', max_na_rm),
    powermax=('power', max_na_rm)
).reset_index()
result

,ID,HRmax,VO2max_abs,powermax
0,1,192.307692,1.759332,170
1,2,183.841055,1.934530,140
2,3,180.303094,3.414079,305
3,4,192.307692,3.437170,305
4,5,188.009416,3.355224,275
5,6,183.378500,3.395895,245
6,7,187.048758,1.635855,170
7,8,184.744806,1.964849,170
8,9,189.873418,1.731755,200
9,10,190.847128,2.347282,200


In [49]:
data_info_clean

,ID,age,Weight,Height,P_vt1,P_vt2,sport
0,1,15,53.7,161,50,140,fencing
1,2,15,56.2,162,80,127,fencing
2,3,16,89.9,191,155,300,fencing
3,4,16,76.5,180,215,275,fencing
4,5,17,76.4,186,110,172,fencing
5,6,17,65.6,169,131,186,fencing
6,7,15,49.8,165,65,108,fencing
7,8,15,51.2,165,80,133,fencing
8,9,14,57.1,176,50,125,fencing
9,10,14,58.8,177,101,162,fencing


In [50]:
# Merging two data frames data_info_clean and calculated HRmax and VO2 max_abs with Powermax
data_info_final = pd.merge(data_info_clean,result,on=['ID'])
data_info_final

,ID,age,Weight,Height,P_vt1,P_vt2,sport,HRmax,VO2max_abs,powermax
0,1,15,53.7,161,50,140,fencing,192.307692,1.741614,170
1,2,15,56.2,162,80,127,fencing,183.579848,1.905661,140
2,3,16,89.9,191,155,300,fencing,179.508212,3.402803,305
3,4,16,76.5,180,215,275,fencing,192.307692,3.424734,305
4,5,17,76.4,186,110,172,fencing,187.533181,3.352790,275
5,6,17,65.6,169,131,186,fencing,182.949874,3.373193,245
6,7,15,49.8,165,65,108,fencing,186.846996,1.600935,170
7,8,15,51.2,165,80,133,fencing,183.910536,1.950244,170
8,9,14,57.1,176,50,125,fencing,189.534358,1.724786,200
9,10,14,58.8,177,101,162,fencing,190.916678,2.330225,200


In [64]:
# Calculating VO2 max by divinding max of VO2 abs and weight of the ID.
# R
data_info_final['VO2_Max'] = data_info_final['VO2max_abs']/ data_info_clean['Weight']
data_info_final['VO2_Max'] = data_info_final['VO2_Max'] * 1000
data_info_final.drop('VO2max_abs',axis=1)
data_info_final

,ID,age,Weight,Height,P_vt1,P_vt2,sport,HRmax,VO2max_abs,powermax,VO2_Max
0,1,15,53.7,161,50,140,fencing,192.307692,1.741614,170,32.432288
1,2,15,56.2,162,80,127,fencing,183.579848,1.905661,140,33.908567
2,3,16,89.9,191,155,300,fencing,179.508212,3.402803,305,37.850975
3,4,16,76.5,180,215,275,fencing,192.307692,3.424734,305,44.767764
4,5,17,76.4,186,110,172,fencing,187.533181,3.352790,275,43.884688
5,6,17,65.6,169,131,186,fencing,182.949874,3.373193,245,51.420627
6,7,15,49.8,165,65,108,fencing,186.846996,1.600935,170,32.147290
7,8,15,51.2,165,80,133,fencing,183.910536,1.950244,170,38.090700
8,9,14,57.1,176,50,125,fencing,189.534358,1.724786,200,30.206405
9,10,14,58.8,177,101,162,fencing,190.916678,2.330225,200,39.629683


In [66]:
data_info_final.to_excel("C:/Users/deepak/Desktop/Datathon/Analysis_Info.xlsx")

In [67]:
data_test.to_excel("C:/Users/deepak/Desktop/Datathon/Analysis_test.xlsx")